In [1]:
import torch
from torch import nn
import numpy as np
from tqdm import tqdm

from unet_dac import UnetDAC

In [2]:
L = 96
K = 256
M = 10 # tmp
INPUT_LEN = 1337
model = UnetDAC(L=L, K=K, M=M)

In [3]:
inputs = torch.Tensor(np.random.random((INPUT_LEN, 2*(M - 1), K, L)))
outputs = torch.Tensor(np.random.randint(0, M, INPUT_LEN))
print(f'Input shape: {inputs.shape}')
res = model(inputs)
print(f"res shape: {res.shape}")

Input shape: torch.Size([1337, 18, 256, 96])


In [5]:
from torch.utils.data import DataLoader, TensorDataset
NUM_TRAIN_EPOCHS = 100
lr: float = 0.001
epochs: int = 100
early_stopping: int = 3
mininbatch_size: int = 64

dataset = TensorDataset(inputs, outputs)
train_loader = DataLoader(dataset, batch_size=mininbatch_size, shuffle=True)



dataloader = torch.utils.data.DataLoader(dataset, batch_size=mininbatch_size, shuffle=True, num_workers=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for epoch in range(epochs):
    for i, minibatch in tqdm(enumerate(train_loader)):
        raw_audio, directions = minibatch

        # Forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(raw_audio)
        loss = criterion(outputs, directions)
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0


c:\Users\agadi\miniconda3\envs\audio_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
0it [00:01, ?it/s]


RuntimeError: Given groups=1, weight of size [128, 256, 3, 3], expected input[128, 128, 32, 12] to have 256 channels, but got 128 channels instead